<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/gluonTS_lagged_value_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 환경 구현 @ CPU Runtime

MXNET 설치 
- GluonTS는 Amazon이 2019년 공개한 Tool로서 MXNET을 backbone으로 사용
  - https://aws.amazon.com/blogs/machine-learning/creating-neural-time-series-models-with-gluon-time-series/

- 어떤 버전의 MXNET을 설치해야 하는 지는 아래 링크 참고
  - https://mxnet.apache.org/get_started/?platform=linux&language=python&processor=gpu&environ=pip&

In [0]:
!pip install mxnet

GluonTS를 설치함

In [0]:
!pip install gluonts

### lag.py 

https://github.com/awslabs/gluon-ts/blob/master/src/gluonts/time_feature/lag.py

In [0]:
from pandas.tseries.frequencies import to_offset
import numpy as np

In [0]:
def _make_lags(middle: int, delta: int) -> np.ndarray:
    """
    Create a set of lags around a middle point including +/- delta
    """
    return np.arange(middle - delta, middle + delta + 1).tolist()

In [0]:
# Lags are target values at the same `season` (+/- delta) but in the previous cycle.
def _make_lags_for_minute(multiple, num_cycles=3):
    # We use previous ``num_cycles`` hours to generate lags
    return [
        _make_lags(k * 60 // multiple, 2) for k in range(1, num_cycles + 1)
    ]

def _make_lags_for_hour(multiple, num_cycles=7):
    # We use previous ``num_cycles`` days to generate lags
    return [
        _make_lags(k * 24 // multiple, 1) for k in range(1, num_cycles + 1)
    ]

def _make_lags_for_day(multiple, num_cycles=4):
    # We use previous ``num_cycles`` weeks to generate lags
    # We use the last month (in addition to 4 weeks) to generate lag.
    return [
        _make_lags(k * 7 // multiple, 1) for k in range(1, num_cycles + 1)
    ] + [_make_lags(30 // multiple, 1)]

def _make_lags_for_week(multiple, num_cycles=3):
    # We use previous ``num_cycles`` years to generate lags
    # Additionally, we use previous 4, 8, 12 weeks
    return [
        _make_lags(k * 52 // multiple, 1) for k in range(1, num_cycles + 1)
    ] + [[4 // multiple, 8 // multiple, 12 // multiple]]

def _make_lags_for_month(multiple, num_cycles=3):
    # We use previous ``num_cycles`` years to generate lags
    return [
        _make_lags(k * 12 // multiple, 1) for k in range(1, num_cycles + 1)
    ]

In [0]:
freq_str = '5min'

In [0]:
# multiple, granularity = get_granularity(freq_str)
offset = to_offset(freq_str)# pandas 함수, to_offset('5min') -> <5 * Minutes>

In [0]:
lags = (
            _make_lags_for_minute(offset.n)
            + _make_lags_for_hour(offset.n / 60.0)
            + _make_lags_for_day(offset.n / (60.0 * 24))
            + _make_lags_for_week(offset.n / (60.0 * 24 * 7))
        )

In [0]:
lags

[[10, 11, 12, 13, 14],
 [22, 23, 24, 25, 26],
 [34, 35, 36, 37, 38],
 [287.0, 288.0, 289.0],
 [575.0, 576.0, 577.0],
 [863.0, 864.0, 865.0],
 [1151.0, 1152.0, 1153.0],
 [1439.0, 1440.0, 1441.0],
 [1727.0, 1728.0, 1729.0],
 [2015.0, 2016.0, 2017.0],
 [2015.0, 2016.0, 2017.0],
 [4031.0, 4032.0, 4033.0],
 [6047.0, 6048.0, 6049.0],
 [8063.0, 8064.0, 8065.0],
 [8639.0, 8640.0, 8641.0],
 [104831.0, 104832.0, 104833.0],
 [209663.0, 209664.0, 209665.0],
 [314495.0, 314496.0, 314497.0],
 [8064.0, 16128.0, 24192.0]]

하나씩 뜯어보기

In [0]:
def _make_lags_for_minute(multiple, num_cycles=3):
    # We use previous ``num_cycles`` hours to generate lags
    return [
        _make_lags(k * 60 // multiple, 2) for k in range(1, num_cycles + 1)
    ]

print (offset.n) # 단위: 분
print ([(k * 60 // offset.n, 2) for k in range(1, 3 + 1)])
print ()

print (_make_lags_for_minute(offset.n)) # 한 시간 간격으로 세 시간동안 같은 분의 앞 뒤 2개씩 더 보겠다.

5
[(12, 2), (24, 2), (36, 2)]

[[10, 11, 12, 13, 14], [22, 23, 24, 25, 26], [34, 35, 36, 37, 38]]


In [0]:
def _make_lags_for_hour(multiple, num_cycles=7):
    # We use previous ``num_cycles`` days to generate lags
    return [
        _make_lags(k * 24 // multiple, 1) for k in range(1, num_cycles + 1)
    ]
multiple = offset.n / 60.0 #(h)
num_cycles=7
print (multiple)
print ([(k * 24 // multiple, 1) for k in range(1, num_cycles + 1)])
print ([((k * 5 * 24 // multiple)/(60*24)  , 1) for k in range(1, num_cycles + 1)])
_make_lags_for_hour(offset.n / 60.0) # 1주일 동안(1일전~7일전) 같은 시간을 중심으로 앞 뒤로 한개를 보겠다. 

0.08333333333333333
[(288.0, 1), (576.0, 1), (864.0, 1), (1152.0, 1), (1440.0, 1), (1728.0, 1), (2016.0, 1)]
[(1.0, 1), (2.0, 1), (3.0, 1), (4.0, 1), (5.0, 1), (6.0, 1), (7.0, 1)]


[[287.0, 288.0, 289.0],
 [575.0, 576.0, 577.0],
 [863.0, 864.0, 865.0],
 [1151.0, 1152.0, 1153.0],
 [1439.0, 1440.0, 1441.0],
 [1727.0, 1728.0, 1729.0],
 [2015.0, 2016.0, 2017.0]]

In [0]:
def _make_lags_for_day(multiple, num_cycles=4):
    # We use previous ``num_cycles`` weeks to generate lags
    # We use the last month (in addition to 4 weeks) to generate lag.
    return [
        _make_lags(k * 7 // multiple, 1) for k in range(1, num_cycles + 1)
    ] + [_make_lags(30 // multiple, 1)]

multiple = offset.n / (60.0 * 24)
num_cycles=4
print (multiple)
print ([(k * 7 // multiple, 1) for k in range(1, num_cycles + 1)])   

_make_lags_for_day(offset.n / (60.0 * 24)) # 1주일 전 ~ 4주일 전, 30일 전 총 5개에 대해 같은 시간대 앞뒤로 하나씩 살펴 보겠다. 

0.003472222222222222
[(2016.0, 1), (4032.0, 1), (6048.0, 1), (8064.0, 1)]


[[2015.0, 2016.0, 2017.0],
 [4031.0, 4032.0, 4033.0],
 [6047.0, 6048.0, 6049.0],
 [8063.0, 8064.0, 8065.0],
 [8639.0, 8640.0, 8641.0]]

In [0]:
def _make_lags_for_week(multiple, num_cycles=3):
    # We use previous ``num_cycles`` years to generate lags
    # Additionally, we use previous 4, 8, 12 weeks
    return [
        _make_lags(k * 52 // multiple, 1) for k in range(1, num_cycles + 1)
    ] + [[4 // multiple, 8 // multiple, 12 // multiple]]

multiple = offset.n / (60.0 * 24 * 7)
num_cycles=3
print (multiple)
print ([(k * 52 // multiple, 1) for k in range(1, num_cycles + 1)])  

_make_lags_for_week(offset.n / (60.0 * 24 * 7)) # 약 1년전(52주전(364일전)), 약 2년전, 약 3년전 동일 시간 대와 앞 뒤하나씩 더 보고, 4주전, 8주전, 12주전 동일시간대도 본다.

0.000496031746031746
[(104832.0, 1), (209664.0, 1), (314496.0, 1)]


[[104831.0, 104832.0, 104833.0],
 [209663.0, 209664.0, 209665.0],
 [314495.0, 314496.0, 314497.0],
 [8064.0, 16128.0, 24192.0]]

In [0]:
lags = (
            _make_lags_for_minute(offset.n) # 전시간부터 3시간 전까지 3개
            + _make_lags_for_hour(offset.n / 60.0) # 전날부터 7일전까지 7개
            + _make_lags_for_day(offset.n / (60.0 * 24)) # 전주부터 4주전까지 4개 + 30일 전 1개 총 5개
            + _make_lags_for_week(offset.n / (60.0 * 24 * 7)) # 얼추 전년부터 얼추 3년전까지 3개 + (4주,8주,12주전) 1개 총 4개
        )
print (len(lags))
lags

19


[[10, 11, 12, 13, 14],
 [22, 23, 24, 25, 26],
 [34, 35, 36, 37, 38],
 [287.0, 288.0, 289.0],
 [575.0, 576.0, 577.0],
 [863.0, 864.0, 865.0],
 [1151.0, 1152.0, 1153.0],
 [1439.0, 1440.0, 1441.0],
 [1727.0, 1728.0, 1729.0],
 [2015.0, 2016.0, 2017.0],
 [2015.0, 2016.0, 2017.0],
 [4031.0, 4032.0, 4033.0],
 [6047.0, 6048.0, 6049.0],
 [8063.0, 8064.0, 8065.0],
 [8639.0, 8640.0, 8641.0],
 [104831.0, 104832.0, 104833.0],
 [209663.0, 209664.0, 209665.0],
 [314495.0, 314496.0, 314497.0],
 [8064.0, 16128.0, 24192.0]]

In [0]:
lag_ub = 1200
# 7보다 크고(뒤에 다 때려 넣을 거라), lag_ub보다 작거나 같은 값만 쓰겠다.
lags = [
        int(lag) for sub_list in lags for lag in sub_list if 7 < lag <= lag_ub
    ]
print (lags)

[10, 11, 12, 13, 14, 22, 23, 24, 25, 26, 34, 35, 36, 37, 38, 287, 288, 289, 575, 576, 577, 863, 864, 865, 1151, 1152, 1153]


In [0]:
lags = [1, 2, 3, 4, 5, 6, 7] + sorted(list(set(lags)))
print (lags)

[1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 22, 23, 24, 25, 26, 34, 35, 36, 37, 38, 287, 288, 289, 575, 576, 577, 863, 864, 865, 1151, 1152, 1153]


In [0]:
num_lags= None
lags[:num_lags]
print (lags)

[1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 22, 23, 24, 25, 26, 34, 35, 36, 37, 38, 287, 288, 289, 575, 576, 577, 863, 864, 865, 1151, 1152, 1153]


### get_lagged_subseuences 

https://github.com/awslabs/gluon-ts/blob/d5a52c76800cf85283d488047fbe096866475064/src/gluonts/model/deepar/_network.py

In [0]:
# Standard library imports
import numpy as np
from typing import List, Optional, Tuple

# Third-party imports
import mxnet as mx
from gluonts.model.common import Tensor
def get_lagged_subsequences(
    F,
    sequence: Tensor,
    sequence_length: int,
    indices: List[int],
    subsequences_length: int = 1,
) -> Tensor:
    """
    Returns lagged subsequences of a given sequence.
    Parameters
    ----------
    sequence : Tensor
        the sequence from which lagged subsequences should be extracted.
        Shape: (N, T, C).
    sequence_length : int
        length of sequence in the T (time) dimension (axis = 1).
    indices : List[int]
        list of lag indices to be used.
    subsequences_length : int
        length of the subsequences to be extracted.
    Returns
    --------
    lagged : Tensor
        a tensor of shape (N, S, C, I), where S = subsequences_length and
        I = len(indices), containing lagged subsequences. Specifically,
        lagged[i, j, :, k] = sequence[i, -indices[k]-S+j, :].
    """
    # we must have: sequence_length - lag_index - subsequences_length >= 0
    # for all lag_index, hence the following assert
    assert max(indices) + subsequences_length <= sequence_length, (
        f"lags cannot go further than history length, "
        f"found lag {max(indices)} while history length is only "
        f"{sequence_length}"
    )
    assert all(lag_index >= 0 for lag_index in indices)

    lagged_values = []
    for lag_index in indices:
        begin_index = -lag_index - subsequences_length
        end_index = -lag_index if lag_index > 0 else None
        lagged_values.append(
            F.slice_axis(
                sequence, axis=1, begin=begin_index, end=end_index
            )
        )

    return F.stack(*lagged_values, axis=-1)

In [0]:
sequence = mx.nd.array([[0,1,2,3,4,5,6,7,8,9]]) # history_length로 다 땡겨 온 target value
sequence_length = 10
lags_seq=[3, 5] # 3개전, 5개전 값을 covariates로 쓰겠다.
subsequences_length = 5 # 모델 학습에 필요한 데이터 집합의 수 (context_length or context_length+predcition_length)

In [0]:
feat_static_cat = mx.nd.zeros((1,10)) # 다음 셀에서 F 선언하려고 일단 만듦
print (feat_static_cat.shape)
feat_static_cat

(1, 10)



[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 1x10 @cpu(0)>

In [0]:
from gluonts.distribution.distribution import getF
F = getF(feat_static_cat)

In [0]:
lags = get_lagged_subsequences(
            F=F,
            sequence=sequence,
            sequence_length=sequence_length,
            indices=lags_seq,
            subsequences_length=subsequences_length,
        )

In [0]:
lagged_values = []
indices = lags_seq
for lag_index in indices:
    begin_index = -lag_index - subsequences_length
    end_index = -lag_index if lag_index > 0 else None
    lagged_values.append(
        F.slice_axis(
            sequence, axis=1, begin=begin_index, end=end_index
        )
    )

In [0]:
lagged_values
# 제일 뒤 쪽 subsequeces_length에 해당하믄 [5,6,7,8,9]가 target value z 가 되고
####### lag_index로 넘겨받은 3번째 전 값들 [2,3,4,5,6]이 첫번째 covariate x1
####### lag_index로 넘겨받은 5번째 전 값들 [0,1,2,3,4]가 두번째 covariate x2가 되는 것 같음

[
 [[2. 3. 4. 5. 6.]]
 <NDArray 1x5 @cpu(0)>, 
 [[0. 1. 2. 3. 4.]]
 <NDArray 1x5 @cpu(0)>]

In [0]:
print (sequence)
print (begin_index)
print (end_index)
print (lagged_values)


[[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]]
<NDArray 1x10 @cpu(0)>
-8
-3
[
[[2. 3. 4. 5. 6.]]
<NDArray 1x5 @cpu(0)>]


In [0]:
F.stack(*lagged_values, axis=-1)
# 마지막 반환이 rnn 구조에서 순차적으로 입력되기 위해 shape이 바뀌게 됨


[[[2. 0.]
  [3. 1.]
  [4. 2.]
  [5. 3.]
  [6. 4.]]]
<NDArray 1x5x2 @cpu(0)>

### gluonts에서 공식적으로 제공하는 lag test code
https://github.com/awslabs/gluon-ts/blob/ed847c9895fb2a225eed0596073a98babd3f31c6/test/model/deepar/test_deepar_lags.py

In [0]:
import itertools

# Third-party imports
import mxnet as mx

# First-party imports
from gluonts.model.deepar._network import DeepARTrainingNetwork

N = 8
T = 96
C = 2
lags = [1, 2, 3, 24, 48]
I = len(lags)
sequence = mx.nd.random.normal(shape=(N, T, C))
S = 48

# (batch_size, sub_seq_len, target_dim, num_lags)
lagged_subsequences = DeepARTrainingNetwork.get_lagged_subsequences(
    F=mx.nd,
    sequence=sequence,
    sequence_length=sequence.shape[1],
    indices=lags,
    subsequences_length=S,
)

assert (N, S, C, I) == lagged_subsequences.shape

# checks that lags value behave as described as in the get_lagged_subsequences contract
for i, j, k in itertools.product(range(N), range(S), range(I)):
    assert (
        (
            lagged_subsequences[i, j, :, k]
            == sequence[i, -lags[k] - S + j, :]
        )
        .asnumpy()
        .all()
    )

In [0]:
lagged_subsequences


[[[[-0.42457297  0.6781736   1.0406365   0.65479124  2.2122064 ]
   [-0.65036863 -0.8431705  -0.30703458 -0.45481315  0.7740038 ]]

  [[-1.4775571  -0.42457297  0.6781736   0.32510808  1.0434403 ]
   [ 0.5587636  -0.65036863 -0.8431705  -1.3002341   1.1839255 ]]

  [[-0.6481019  -1.4775571  -0.42457297  0.3679345   1.8917114 ]
   [ 0.07313449  0.5587636  -0.65036863  1.4534262  -1.2347414 ]]

  ...

  [[ 0.8258905  -2.0099306  -0.11363573  1.9676613   1.0406365 ]
   [ 1.0249989   1.2595973  -0.61025107 -0.00440959 -0.30703458]]

  [[ 0.5808451   0.8258905  -2.0099306   2.6056926   0.6781736 ]
   [ 0.486164    1.0249989   1.2595973   0.58565795 -0.8431705 ]]

  [[-1.4802811   0.5808451   0.8258905  -0.6968178  -0.42457297]
   [ 1.4363087   0.486164    1.0249989   0.15026927 -0.65036863]]]


 [[[-0.52480155  1.6491317  -1.436299   -0.28338912 -0.75775224]
   [ 0.3005414  -1.2902275  -1.2416507   0.9629334   0.81197935]]

  [[-1.4925312  -0.52480155  1.6491317  -1.1144744  -0.66640246]
 